# week 8  과제
## 컴퓨터 과학과 201710957 이유진

## 연습문제



```greeter```의 ```greeting```의 데이터타입을 ```string```에서 ```bytes32```로 변경하세요.

- ```string```에서 ```bytes32```로 변경하면 gas의 차이가 있는지 확인

- 노드에서 배포

- 노드에서 인사를 설정하고 ```setGreeting("Hello World!")```, ```greet()```을 출력하세요.



## 1) Solidity 코딩

In [1]:
%%writefile C:\Users\sofod\Code\201710957\src\greeter.sol
pragma solidity ^0.4.0;

contract Gretter {
    string greeting;
    
    constructor() public {
        greeting = 'Hello';
    }
    
    function setGreeting( string memory _greeeting) public {
        greeting = _greeeting;
    }
    
    function greet() view public returns (string memory){
        return greeting;
    }
}

Writing C:\Users\sofod\Code\201710957\src\greeter.sol


In [126]:
%%writefile C:\Users\sofod\Code\201710957\src\greeter2.sol
pragma solidity ^0.4.0;
contract Gretter2 {
    bytes32 greeting;
    
    constructor() public {
        greeting = 'Hello';
    }
    
    function setGreeting( bytes32  _greeeting) public {
        greeting = _greeeting;
    }
    
    function greet() view public returns (bytes32 ){
        return greeting;
    }
}

Overwriting C:\Users\sofod\Code\201710957\src\greeter2.sol


## 2) Solc를 이용하여 컴파일

In [127]:
!solc --abi --bin  C:\Users\sofod\Code\201710957\src\greeter2.sol


======= C:\Users\sofod\Code\201710957\src\greeter2.sol:Gretter2 =======
Binary: 
608060405234801561001057600080fd5b507f48656c6c6f0000000000000000000000000000000000000000000000000000006000816000191690555060ef8061004a6000396000f3006080604052600436106049576000357c0100000000000000000000000000000000000000000000000000000000900463ffffffff16806350513b4f14604e578063cfae321714607c575b600080fd5b348015605957600080fd5b50607a600480360381019080803560001916906020019092919050505060ac565b005b348015608757600080fd5b50608e60ba565b60405180826000191660001916815260200191505060405180910390f35b806000816000191690555050565b600080549050905600a165627a7a7230582083295188cd0d51d9beb9d50e027b8b4c1814fad34c9d1d8885378dcc17e5f6b60029
Contract JSON ABI 
[{"constant":false,"inputs":[{"name":"_greeeting","type":"bytes32"}],"name":"setGreeting","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"greet","outputs":[{"name":"","type":"bytes32"}],"payable":false,"s

### gas 차이 확인

In [128]:
!solc --gas C:\Users\sofod\Code\201710957\src\greeter2.sol


======= C:\Users\sofod\Code\201710957\src\greeter2.sol:Gretter2 =======
Gas estimation:
construction:
   20122 + 47800 = 67922
external:
   greet():	442
   setGreeting(bytes32):	20223


In [129]:
!solc --gas C:\Users\sofod\Code\201710957\src\greeter.sol


======= C:\Users\sofod\Code\201710957\src\greeter.sol:Gretter =======
Gas estimation:
construction:
   infinite + 145400 = infinite
external:
   greet():	infinite
   setGreeting(string):	infinite


string을 사용하였을때는 infinite로 산정불가능이었으나 bytes32로 변경하였을때 글자수가 제한되어 gas비가 산정되어있음을 알수있다. 

## 3) node 를 이용한 배포

In [6]:
!geth --exec "eth.coinbase" attach http://localhost:8445

"0x478e5deacb2c8b60452f7d9822e50643bb78c1d3"


In [7]:
%%writefile C:\Users\sofod\Code\201710957\src\greeting_deploy.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));
var shelloContract = new web3.eth.Contract([{"constant":false,"inputs":[{"name":"_greeeting","type":"bytes32"}],"name":"setGreeting","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"greet","outputs":[{"name":"","type":"bytes32"}],"payable":false,"stateMutability":"view","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}]);
shelloContract
    .deploy({
            data: '0x608060405234801561001057600080fd5b507f48656c6c6f0000000000000000000000000000000000000000000000000000006000816000191690555060ef8061004a6000396000f3006080604052600436106049576000357c0100000000000000000000000000000000000000000000000000000000900463ffffffff16806350513b4f14604e578063cfae321714607c575b600080fd5b348015605957600080fd5b50607a600480360381019080803560001916906020019092919050505060ac565b005b348015608757600080fd5b50608e60ba565b60405180826000191660001916815260200191505060405180910390f35b806000816000191690555050565b600080549050905600a165627a7a7230582083295188cd0d51d9beb9d50e027b8b4c1814fad34c9d1d8885378dcc17e5f6b60029', 
    })
    .send({
     from: "0x478e5deacb2c8b60452f7d9822e50643bb78c1d3",
     gas: '4700000'
    }, function (error, transactionHash){ 
            console.log("error :" + error);
            console.log("hash : " + transactionHash); 
    })
    .then(function(newContractInstance){
        console.log("address : " + newContractInstance.options.address)
    });

Overwriting C:\Users\sofod\Code\201710957\src\greeting_deploy.js


In [8]:
!node C:\Users\sofod\Code\201710957\src\greeting_deploy.js

error :null
hash : 0x8fe25653dc036f7c8aaea9764931eb4e26a0e20c01dd1f9c44155cbf25969d53
address : 0xFb97a0c88e78D0a2D37079c7F4A141246657AeEb


## 4) API 사용 contractAddress 호출하여 사용

In [9]:
%%writefile C:\Users\sofod\Code\201710957\src\greeterUse.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));
var _abiArray=[{"constant":false,"inputs":[{"name":"_greeeting","type":"bytes32"}],"name":"setGreeting","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"greet","outputs":[{"name":"","type":"bytes32"}],"payable":false,"stateMutability":"view","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
var greeter = new web3.eth.Contract(_abiArray,"0xFb97a0c88e78D0a2D37079c7F4A141246657AeEb");

greeter.methods.greet().call().then(function(value) {console.log(web3.utils.toAscii(value));});

var txt = web3.utils.fromAscii("Hello World");
greeter.methods.setGreeting(txt).send({from:"0x478e5deacb2c8b60452f7d9822e50643bb78c1d3",gas:100000});
greeter.methods.greet().call().then(function(value) {console.log(web3.utils.toAscii(value));});



Overwriting C:\Users\sofod\Code\201710957\src\greeterUse.js


In [10]:
!node C:\Users\sofod\Code\201710957\src\greeterUse.js

Hello                           
Hello                           


In [11]:
!node C:\Users\sofod\Code\201710957\src\greeterUse.js

Hello World                     
Hello World                     


async 한 특성 때문에, 한번더 실행했을경우에 Hello World 로 변경되어 출력됨을 알수 있다,

# 연습문제

어떤 수에 7을 곱하는 스마트 컨트랙을 만든다.

컨트랙은 ```Multiply7```, 함수는:

function multiply(uint input) public pure returns (uint)

컨트랙은 Solidity로, 클라이언트는 노드로 프로그램한다.

- 8을 인자로 너기면, 7을 곱해서 56을 출력하세요.

## 1) Solidity 코딩

In [81]:
%%writefile C:\Users\sofod\Code\201710957\src\Multiply.sol
pragma solidity ^0.4.0;

contract Multiply7{
    function multiply(uint input) public pure returns (uint){
        uint num = input * 7;
        return num;
    }
   
}

Overwriting C:\Users\sofod\Code\201710957\src\Multiply.sol


## 2) Solc를 이용하여 컴파일

In [82]:
!solc --abi  C:\Users\sofod\Code\201710957\src\Multiply.sol


======= C:\Users\sofod\Code\201710957\src\Multiply.sol:Multiply7 =======
Contract JSON ABI 
[{"constant":true,"inputs":[{"name":"input","type":"uint256"}],"name":"multiply","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"pure","type":"function"}]


In [83]:
!solc --bin  C:\Users\sofod\Code\201710957\src\Multiply.sol


======= C:\Users\sofod\Code\201710957\src\Multiply.sol:Multiply7 =======
Binary: 
608060405234801561001057600080fd5b5060c08061001f6000396000f300608060405260043610603f576000357c0100000000000000000000000000000000000000000000000000000000900463ffffffff168063c6888fa1146044575b600080fd5b348015604f57600080fd5b50606c600480360381019080803590602001909291905050506082565b6040518082815260200191505060405180910390f35b600080600783029050809150509190505600a165627a7a72305820fc0360d2e7d5742c067e77981d7b4f8951eaa1ec6256566c444ac33d53b630090029


## 3) node 를 이용한 배포

In [12]:
!geth --exec "eth.coinbase" attach http://localhost:8445

"0x478e5deacb2c8b60452f7d9822e50643bb78c1d3"


In [13]:
%%writefile C:\Users\sofod\Code\201710957\src\multiply_deploy.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));
var shelloContract = new web3.eth.Contract(
[{"constant":true,"inputs":[{"name":"input","type":"uint256"}],"name":"multiply","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"pure","type":"function"}]
);

shelloContract
    .deploy({
            data: '0x608060405234801561001057600080fd5b5060c08061001f6000396000f300608060405260043610603f576000357c0100000000000000000000000000000000000000000000000000000000900463ffffffff168063c6888fa1146044575b600080fd5b348015604f57600080fd5b50606c600480360381019080803590602001909291905050506082565b6040518082815260200191505060405180910390f35b600080600783029050809150509190505600a165627a7a72305820fc0360d2e7d5742c067e77981d7b4f8951eaa1ec6256566c444ac33d53b630090029', 
    })
    .send({
     from: "0x478e5deacb2c8b60452f7d9822e50643bb78c1d3",
     gas: '4700000'
    }, function (error, transactionHash){ 
            console.log("error :" + error);
            console.log("hash : " + transactionHash); 
    })
    .then(function(newContractInstance){
        console.log("address : " + newContractInstance.options.address)
    });

Overwriting C:\Users\sofod\Code\201710957\src\multiply_deploy.js


In [14]:
!node C:\Users\sofod\Code\201710957\src\multiply_deploy.js

error :null
hash : 0x55bf65bcc75a22ca61620d4a01cc533c2d40314a2c203f5242889ce0eeeea38d
address : 0x303B2Ec2dB3105C332C2A802DB0ecb9b5370eC92


## 4) API 사용 contractAddress 호출하여 사용

In [17]:
%%writefile C:\Users\sofod\Code\201710957\src\multiplyUse.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8445"));
var _abiArray=[{"constant":true,"inputs":[{"name":"input","type":"uint256"}],"name":"multiply","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"pure","type":"function"}];
var multiplyer = new web3.eth.Contract(_abiArray,"0x303B2Ec2dB3105C332C2A802DB0ecb9b5370eC92");

multiplyer.methods.multiply(8).call().then(function(val) { console.log(val);})


Overwriting C:\Users\sofod\Code\201710957\src\multiplyUse.js


In [18]:
!node C:\Users\sofod\Code\201710957\src\multiplyUse.js

56
